In [11]:
import numpy as np
import pandas as pd
import re

In [185]:
with open('../input/day12.txt', 'r') as f:
    lines = f.read().splitlines()

lines[:5]

['AAAAAAMMMMMMMMMMMMMMMMMMCCCCCCCCACAYYYYYYYYYYYYYYYYWWJWWIIIIIIIIIIIIIIIIIIIIIFVFMMMMMMMMMYRRYRRRRRNXXNAANIIIIIEEEEEEEEEEEEEEERRRRRRRRQQQQQCQ',
 'AAAAAAAAMSMMMMMMMMMMMMMMMCCCCCCCCCYYYYYYYYYYYYYYYYYWWWWWWWIIIIIIIIIIIIIIIIIIFFFFFFMMMMMMMYYRYRRNNNNNNNNNNNNNIEEEEEEEEEEEEEEERRRRRRRFQQQQQQQQ',
 'AAAAAAAFSSSMMMMMMMMMMMMXMCCCCCCCCCUCYYYYYYYYYYYYYYYWWWWWWWWIIIIIIIIIIIIIIIIIFFFFFFFMMMMYYYYYYYYNNNNNNNNNNNNIIIIEEEEEEEEEEEEERRRRRRRQQQQQQQQQ',
 'AAAAAASSSSSSMMMMMMMMMMMXXXCCCCCCCCCCYYYYYYYYYYYYYYYYWWWWWWWIIIIIIIIIIIIIIIIFFFFFFFIMMMYYYYYYYYNNNNNNNNNNNNNNIIIIIEEEEEEEEEEEERRRRRRQQQQQQQQQ',
 'AAAAASSSSSSSSMMMMMMMMMXXXXXCCCCCCPCCYYYYYYYYYYYYYYYYWWWWWWWWIPPIIIIIIHHHIHIFFFFFFFIMMMYYYYYYYYNNNNNNNNNNNNNNNNNIEEEEEEEEEEEERRRRRRROQQQQQQQQ']

In [186]:
def get_neighbors(coord):
    x, y = coord[0], coord[1]
    return (
        [x+1, y],
        [x-1, y],
        [x, y+1],
        [x, y-1],
    )

In [187]:
def is_in_bounds(coord):
    return coord[0] >= 0 and coord[0] < len(lines) and coord[1] >= 0 and coord[1] < len(lines[0])

In [188]:
def find_boundaries(x, y, region_type, tmp_reg):
    area = 1
    perim = 0
    tmp_reg[x] = tmp_reg[x][:y] + '-' + tmp_reg[x][y+1:]
    regions[x] = regions[x][:y] + '-' + regions[x][y+1:]
    coords = [(x,y)]
    for coord in get_neighbors([x,y]):
        # print(coord)
        if not is_in_bounds(coord):
            perim += 1
            # print('oob')
        elif tmp_reg[coord[0]][coord[1]] != region_type and tmp_reg[coord[0]][coord[1]] != '-':
            perim += 1
            # print('eol')
        elif tmp_reg[coord[0]][coord[1]] == region_type:
            # print('continuing')
            coords.append((coord[0], coord[1]))
            a, p, c = find_boundaries(coord[0], coord[1], region_type, tmp_reg)
            # a, p = 1, 1
            area += a
            perim += p
            coords = coords + c
    
    return area, perim, coords

In [194]:
def get_corner_counts(coords):
    '''
    ugly af
    '''
    ss = set(coords)
    sides = 0
    for s in ss:
        #corners
        # print(s)
        if (s[0]-1, s[1]) not in ss and (s[0], s[1]-1) not in ss:
            sides += 1 
            # print('corner 1')
        if (s[0]-1, s[1]) not in ss and (s[0], s[1]+1) not in ss:
            sides += 1
            # print('corner 2')
        if (s[0]+1, s[1]) not in ss and (s[0], s[1]-1) not in ss:
            sides += 1
            # print('corner 3')
        if (s[0]+1, s[1]) not in ss and (s[0], s[1]+1) not in ss:
            sides += 1
            # print('corner 4')

    #inner corners
        if (s[0]-1, s[1]) in ss and (s[0], s[1]-1) in ss and (s[0]-1, s[1]-1) not in ss:
            # print('icorner 1')
            sides += 1
        if (s[0]-1, s[1]) in ss and (s[0], s[1]+1) in ss and (s[0]-1, s[1]+1) not in ss:
            # print('icorner 2')
            sides += 1
        if (s[0]+1, s[1]) in ss and (s[0], s[1]-1) in ss and (s[0]+1, s[1]-1) not in ss:
            # print('icorner 3')
            sides += 1
        if (s[0]+1, s[1]) in ss and (s[0], s[1]+1) in ss and (s[0]+1, s[1]+1) not in ss:
            # print('icorner 4')
            sides += 1

    return sides

In [190]:
regions = lines.copy()
areas = []
perims = []
sides = []
for x in range(len(lines)):
    for y in range(len(lines[0])):
        if regions[x][y] != '-':
            region_type = regions[x][y]
            tmp_reg = lines.copy()
            a, p, c = find_boundaries(x, y, region_type, tmp_reg)
            s = get_corner_counts(c)
            areas.append(a)
            perims.append(p)
            sides.append(s)
                                
            
            

In [191]:
df = pd.DataFrame([areas, perims, sides]).transpose().assign(
    price = lambda x: x[0]*x[1],
    price_p2 = lambda x: x[0] * x[2],
)
df

,0,1,2,price,price_p2
0,48,38,18,1824,864
1,104,70,42,7280,4368
2,87,66,44,5742,3828
3,1,4,4,4,4
4,1,4,4,4,4
...,...,...,...,...,...
614,1,4,4,4,4
615,7,14,8,98,56
616,7,16,10,112,70
617,16,22,12,352,192


In [192]:
p1 = df['price'].sum()
p1

np.int64(1452678)

In [193]:
p2 = df['price_p2'].sum()
p2

np.int64(873584)